In [1]:
import pandas as pd
import numpy as np

## Features engineering de pageviews

### Carga de datos

In [2]:
pageviews = pd.read_pickle("../data/pageviews_total_formateado.pkl")

---

### Cargo features generados hasta el momento


In [3]:
# Trimestre 2 - 3 - 4 para los que tengo datos anteriores
CANT_TRIMESTRES = 4

In [4]:
ventanas = {}
vent_no_encontradas =[]

In [5]:
print("Cargando features")
for ventana_nro in range(1,CANT_TRIMESTRES) : #Trimestres del 1 al 3
    try:
        ventanas[ventana_nro] = pd.read_csv("../data/features/ventana_{}_pageviews.csv".format(ventana_nro),
                                            dtype = {"USER_ID":"category"},index_col=0)
        
    except FileNotFoundError:
        print("No se encontraron los features, se generarán los dataframe requeridos")
        vent_no_encontradas.append(ventana_nro)
    continue
    
for ventana_nro in vent_no_encontradas:    
    ##Recolecto los features del trimestre anterior al que quiero predecir
    condicion = pageviews["trimestre"] == (ventana_nro)
    
    ventanas[ventana_nro] = pageviews.loc[condicion]["USER_ID"] \
        .drop_duplicates().to_frame().set_index("USER_ID").copy()

Cargando features


### Set para predecir

In [6]:
ventana_predecir = {}

In [7]:
try:
    print("Cargando features para predecir")
    ventana_predecir[4] = pd.read_csv("../data/features/predecir_pageviews.csv")
except FileNotFoundError:
    print("No se encontro el feature deseado, se generara el dataframe requerido")
    ventana_predecir[4] = pd.read_csv("../data/users.csv", dtype = {"USER_ID":"category"})
    ventana_predecir[4].set_index("USER_ID",inplace = True)

Cargando features para predecir


---

### Diccionario de features

In [8]:
features = {}

In [9]:
features_ya_generados = {}

for ventana_nro in range (1, CANT_TRIMESTRES):
    
    features_ya_generados[ventana_nro] = list(ventanas[ventana_nro].columns.values)
    
features_ya_generados[4] = list(ventana_predecir[4].columns.values)



---

### Creación de features

#### Cantidad de paginas vista por un usuario en todo el trimestre

In [10]:
def cantidad_de_paginas_vistas(dataframe):
    return dataframe.groupby("USER_ID").agg({"FEC_EVENT":"count"})\
                    .rename(columns = {"FEC_EVENT":"cantidad_paginas_vistas_trimestre"})

In [11]:
features["cantidad_paginas_vistas_trimestre"] = (cantidad_de_paginas_vistas,["cantidad_paginas_vistas_trimestre"])

#### Cantidad de paginas vista dentro de cada mes dentro del trimestre

In [12]:
def cantidad_paginas_vistas_por_mes(dataframe):
    
    vistas_por_mes = dataframe.groupby(["USER_ID","mes"]).agg({"trimestre":"count"})
    vistas_por_mes = vistas_por_mes.unstack().droplevel(level = 0, axis = 1)
    
    
    cantidad_meses = len(vistas_por_mes.columns)
    nuevas_columnas = [x for x in range(1,cantidad_meses+1)]
    vistas_por_mes.columns = nuevas_columnas    #transformo todos los meses en un rango 1-3
    
    for mes in range(1,cantidad_meses + 1):
        if mes not in list(vistas_por_mes.columns.values):
            vistas_por_mes.insert(dia,dia,0)
            
    vistas_por_mes.columns = ["cant_paginas_vistas_en_mes_{}".format(mes) for mes in range(1,4)]
        
    
    return vistas_por_mes

In [13]:
columnas_generadas = ["cant_paginas_vistas_en_mes_{}".format(mes) for mes in range(1,4)]
features["cant_paginas_vistas_por_mes"] = (cantidad_paginas_vistas_por_mes,columnas_generadas) 

---

## Generacion de features

In [14]:
def generar_feature_en_ventanas(dataframe, generador_feature, destino, nro_ventana):
    """ 
    Esta funcion calcula un feature para un trimestre o ventana en especifico, lo une con los ya calculados
    y lo guarda en el diccionario 'destino'.    
    dataframe: dataframe de datos de donde se calcularan los features, en este caso pageviews
    generador_feature: funcion que calculara el feature y devolvera su resultado
    destino: diccionario que contiene las ventanas de features calculados
    """
    dataframe = dataframe.loc[dataframe["trimestre"] == nro_ventana]
    feature = generador_feature(dataframe)
    destino[nro_ventana] = destino[nro_ventana].merge(feature, left_index = True, right_index= True, how = 'left')

### Genero los features


In [15]:
#Posicion dentro del diccionario de features que ocupa la lista de columnas generadas por el mismo
POS_COLUMNAS_DEL_FEATURE = 1  
#Posicion dentro del diccionario de features que ocupa la funcion que lo genera.
POS_FUNCION_DEL_FEATURE = 0

def generar_features_faltantes(dataframe,destino,nro_ventana):
    """
    Funcion para generar los features de cada trimestre, si solo si no se encuentran generados,
    evita la repeticion de calculo de features si estos ya se encuentran generados.
    
    dataframe: Dataframe a partir de cual se calcularan los features.
    destino: Diccionario de ventanas para los que calcular los features.
    nro_ventana: Numero de ventana a partir de la cual se calculan los features.
    """
    
    for nombre_feature in features:    
        #Obtengo los nombres de las columnas generadas por la funcion que calcula el feature
        #Una funcion puede devolver mas de una columna por lo que obtengo una lista de valores
        columnas_generadas = features[nombre_feature][POS_COLUMNAS_DEL_FEATURE] 
        feature_ya_generado = False
        
        for columna in columnas_generadas: 
            
            #Verifico si los features ya fueron generados o no
            if columna in features_ya_generados[nro_ventana]: 
                feature_ya_generado = True
                break
    
        if feature_ya_generado:
            continue

        #Si no fueron generados, los genero y actualizo el listado:    
        generar_feature_en_ventanas(dataframe, features[nombre_feature][POS_FUNCION_DEL_FEATURE], destino,nro_ventana)                              
        features_ya_generados[nro_ventana].extend(features[nombre_feature][POS_COLUMNAS_DEL_FEATURE])

In [16]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    generar_features_faltantes(pageviews, ventanas, ventana_nro)

generar_features_faltantes(pageviews, ventana_predecir,4)    

In [17]:
ventanas[1].head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3
USER_ID,,,,
7237,862,373.0,290.0,199.0
4913,1350,582.0,362.0,406.0
9796,736,196.0,204.0,336.0
9428,1006,262.0,142.0,602.0
4719,721,197.0,271.0,253.0


### Guardado

In [18]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    ventanas[ventana_nro].to_csv("../data/features/ventana_{}_pageviews.csv".format(ventana_nro))

ventana_predecir[4].to_csv("../data/features/predecir_pageviews.csv")